In [ ]:
!pip install pandas numpy nltk scikit-learn


In [ ]:
nltk.download('stopwords')


In [ ]:
data = pd.read_csv("fake_news.csv")
data.head()



In [ ]:
import os
os.getcwd()


In [7]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

fake['label'] = 0
true['label'] = 1

data = pd.concat([fake, true])
data = data[['text', 'label']]

data.head()


,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [9]:


# 1. Import required libraries
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 2. Load datasets
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

# 3. Add labels
fake['label'] = 0   # Fake news
true['label'] = 1   # Real news

# 4. Combine datasets
data = pd.concat([fake, true], axis=0)
data = data[['text', 'label']]
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle

# 5. Clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    return text

data['text'] = data['text'].apply(clean_text)

# 6. Split dataset
X = data['text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# 7. Convert text to numbers using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 8. Train model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# 9. Predict and evaluate
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# 10. Test with custom news
def predict_news(news):
    news = clean_text(news)
    news_vector = vectorizer.transform([news])
    prediction = model.predict(news_vector)
    return "REAL NEWS 🟢" if prediction[0] == 1 else "FAKE NEWS 🔴"

# Example
print("\nCustom Test:")
print(predict_news("Government announces new education policy for universities"))


Accuracy: 0.9884187082405346

Classification Report:

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5871
           1       0.99      0.99      0.99      5354

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225


Custom Test:
FAKE NEWS 🔴
